# Salt And Paper

Salt and paper is a form of noise sometimes seen on images that have sudden disturbances in the image signal. It is very common on black and white pictures.

## Libraries

For this exercie, the libraries needed are ``cv2``, ``numpy``, ``skimage.metrics`` and ``random``. ``Random`` is installed with Python, while the other three need to be installed from ``pip``.

On the root of the project, run this command:
```sh
pip install -r requirements.txt
```

In [1]:
import cv2
import numpy as np

from skimage.metrics import structural_similarity as ssim
from random import random

## Image

The image gonna be used for this example is the cover art for Phil Collins' first album called Face Value. The image is taken twice, once when the album came out in early 1981 and when the album was reissued in 2015. That means that there will be a difference in image fidelity in both pictures.

![Face Values artwork](./img/face_value.jpg)

In [2]:
image = cv2.imread('img/face_value.jpg', 0)

## Gaussian Blur

It's time to add some blur into the picture!

First, it's needed to apply some filters to the kernel of the image.

In [3]:
# Applying the blur into the picture
kernel = np.ones((5, 5), np.float32) / 25

average_image = cv2.filter2D(image, -1, kernel)
gauss_image = cv2.GaussianBlur(image, (5, 5), 0)

### Displaying the average blurred image 

![Average Blurred image](./img/average.png)

It can be noticed that the picture is more blurred than before. Especially in the left picture that it's newer.

### Displaying the Structural Similarity Index (SSIM) of the blurred picture

**Structural Similarity Index (SSIM)** is the loss of fidelity between the original picture to the compressed copy that's made.

In [4]:
def calc_simil(og_image, image):
    simil_score, _ = ssim(og_image, image, full=True)
    print(f"SSIM score is: {simil_score}")

calc_simil(image, average_image)

SSIM score is: 0.7470682588545584


### Displaying the Guassian picture

In [5]:
calc_simil(image, gauss_image)

SSIM score is: 0.8310405477664277


![Guassian blur](./img/gaus.png)

With this method, there is less blur and the features in Phil Collins' face is more clear.

## Image sharpening

It's time to sharpen the image.

In [6]:
kernel = np.array(
    [
        [0, -1 , 0],
        [-1, 5, -1],
        [0, -1, 0]
    ]
)

sharp_image = cv2.filter2D(image, -1, kernel)
sharp_gauss_image = cv2.filter2D(gauss_image, -1, kernel)

### Average sharp

In [7]:
calc_simil(image, sharp_image)

SSIM score is: 0.5990018162347946


![Average](./img/sharp.png)

The image looks too wrong. It's like Phil Collins has snow on his face.

### Guassian sharp

In [8]:
calc_simil(image, sharp_gauss_image)

SSIM score is: 0.8906897600245756


![Guassian Sharp](./img/sharp_gaus.png)

The image looks sharper but far more natural than it was before.

## Salt and Sault Noise

Now it's time to add some noise to the picture

### Algorithm

This is the algorithm implemented in Python.

In [9]:
def salt_and_paper(image, probability):
    noise_image = np.zeros(image.shape, np.uint8)

    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            rand = random()
            if rand < probability:
                noise_image[row][col] = 0
            elif rand > (1 - probability):
                noise_image[row][col] = 255
            else:
                noise_image[row][col] = image[row][col]
    
    return noise_image

### Adding the noise

In [10]:
noise_image = salt_and_paper(image, 0.12)

calc_simil(image, noise_image)

SSIM score is: 0.09915893011117755


![Adding noise](./img/noise.png)

The picture is not as clear as it once was.

### Trying to remove the noise

To remove the noise, median blur is gonna be used.

In [11]:
median = cv2.medianBlur(noise_image, 5)

calc_simil(image, median)

SSIM score is: 0.7420197044256345


![Median](./img/median.png)

### Comparison with the original

![Median](./img/median.png)

![Original](./img/face_value.jpg)

All the details are lost from the original. The album title cannot be read and it's very difficult to see some of the details in Phil Collins' face.

## Source code

```py
import cv2
import numpy as np

from skimage.metrics import structural_similarity as ssim
from random import random

def show_image(og_image, image, name):
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, image)

    simil_score, _ = ssim(og_image, image, full=True)
    print(f"{name} SSIM score is: {simil_score}")


def salt_and_paper(image, probability):
    noise_image = np.zeros(image.shape, np.uint8)

    for row in range(image.shape[0]):
        for col in range(image.shape[1]):
            rand = random()
            if rand < probability:
                noise_image[row][col] = 0
            elif rand > (1 - probability):
                noise_image[row][col] = 255
            else:
                noise_image[row][col] = image[row][col]
    
    return noise_image


if __name__ == '__main__':
    image = cv2.imread('img/face_value.jpg', 0)

    cv2.namedWindow('Original Image', cv2.WINDOW_NORMAL)
    cv2.imshow('Original Image', image)

    kernel = np.ones((5, 5), np.float32) / 25

    average_image = cv2.filter2D(image, -1, kernel)
    gauss_image = cv2.GaussianBlur(image, (5, 5), 0)

    show_image(image, average_image, 'Average Image')
    show_image(image, gauss_image, 'Gaus Image')

    kernel = np.array(
        [
            [0, -1 , 0],
            [-1, 5, -1],
            [0, -1, 0]
        ]
    )

    sharp_image = cv2.filter2D(image, -1, kernel)
    sharp_gauss_image = cv2.filter2D(gauss_image, -1, kernel)

    show_image(image, sharp_image, 'Sharp Image')
    show_image(image, sharp_gauss_image, 'Sharp Gaus Image')

    noise_image = salt_and_paper(image, 0.12)
    show_image(image, noise_image, 'Noise Image')

    median = cv2.medianBlur(noise_image, 5)
    show_image(image, median, 'Median')

    cv2.waitKey(0)
    cv2.destroyAllWindows()
```